<a href="https://colab.research.google.com/github/Cellous/ai-bootcamp-portfolio/blob/main/week-05-agent-execution-and-memory/notebooks/Week5_Unit2_2_The_Agent_Execution_Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unit 2.2 – Agent Execution Loop (Reason → Act → Reflect)

**Course:** AI Bootcamp / Agents  
**Week:** 5  
**Unit:** 2.2 – The Agent Execution Loop  
**Student:** Marcellous

## Objective
Demonstrate an agent execution loop that follows the **Reason → Act → Reflect** pattern using a local language model and persistent memory.

## How This Agent Works
- **Reason:** Checks user input for memory-related questions (e.g., “What did I say earlier?”)
- **Act:** Either retrieves stored memory or generates a response using a local model
- **Reflect:** Saves conversation history to a local memory file (`ai_memory.txt`)

## Technical Details
- Model: `distilgpt2` (no API key, no cost)
- Interface: Gradio
- Memory: File-based persistence
- Runs fully offline after installation

## Notes
- The model is not instruction-tuned; repetitive outputs are expected.
- The focus of this assignment is the **agent loop structure**, not model quality.


In [ ]:
!pip install -q gradio transformers torch

In [ ]:
import gradio as gr
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# -----------------------------
# 1. Setup model
# -----------------------------
model_name = "sshleifer/tiny-gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Add pad token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

agent = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=-1    # explicitly force CPU
)


In [ ]:
# -----------------------------
# 2. Memory setup
# -----------------------------
MEMORY_FILE = "ai_memory.txt"

if os.path.exists(MEMORY_FILE):
    with open(MEMORY_FILE, "r", encoding="utf-8") as f:
        memory = f.read()
else:
    memory = ""

def save_memory():
    with open(MEMORY_FILE, "w", encoding="utf-8") as f:
        f.write(memory)


In [ ]:
def chat_with_ai(user_message, chat_history):
    global memory

    # -------- REASON --------
    if "what did i say earlier" in user_message.lower():
        if memory.strip():
            reply = f"You previously said: {memory.strip().splitlines()[-2]}"
        else:
            reply = "I don't have any memory yet."

    elif "remember" in user_message.lower():
        reply = "Okay, I will remember that."

    else:
        # -------- ACT --------
        context = f"{memory}\nUser: {user_message}\nAI:"
        result = agent(context, max_new_tokens=80, temperature=0.6)[0]["generated_text"]
        reply = result.split("AI:")[-1].strip()

    # -------- REFLECT --------
    memory += f"\nUser: {user_message}\nAI: {reply}"
    save_memory()

    chat_history.append((user_message, reply))
    return chat_history, ""


In [ ]:
def agent_loop(user_message, chat_history):
    global memory

    # -------------------------
    # REASON
    # -------------------------
    if "what did i say earlier" in user_message.lower():
        if memory.strip():
            reply = f"You previously said: {memory.strip().splitlines()[-2]}"
        else:
            reply = "I don't have any memory yet."

    # -------------------------
    # ACT
    # -------------------------
    else:
        prompt = f"{memory}\nUser: {user_message}\nAI:"
        out = agent(prompt, max_new_tokens=80, temperature=0.6)[0]["generated_text"]
        reply = out.split("AI:")[-1].strip()

    # -------------------------
    # REFLECT (save memory)
    # -------------------------
    memory += f"\nUser: {user_message}\nAI: {reply}"
    save_memory()

    chat_history.append((user_message, reply))
    return chat_history, ""


In [ ]:
# -----------------------------
# 4. Gradio interface
# -----------------------------
with gr.Blocks() as ui:
    gr.Markdown("## Agent Execution Loop Demo (Reason → Act → Reflect)")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Type your message")
    clear = gr.Button("Clear Memory")

    msg.submit(agent_loop, [msg, chatbot], [chatbot, msg])

    def clear_memory():
        global memory
        memory = ""
        open(MEMORY_FILE, "w").close()
        return []

    clear.click(clear_memory, None, chatbot)

ui.launch()
